In [0]:
import argparse
import time
import torch
import torch.nn
from torch.autograd import Variable
import torch.nn as nn
from lm import repackage_hidden, LM_LSTM
import reader
import numpy as np


Ниже - обучение базовой модели из статьи: bigLSTM(small PTB)
Обученная модель сохранена в файле lm_model.pt  
Поэтому это можно скипнуть

In [0]:
criterion = nn.CrossEntropyLoss()
data = 'data'
hidden_size = 1500
num_steps = 35
num_layers = 2
batch_size = 20
num_epochs = 13 
dp_keep_prob = 0.35
inital_lr = 20.0
save = 'lm_model.pt'

In [0]:
def repackage_hidden(h):
  """Wraps hidden states in new Variables, to detach them from their history."""
  

  if type(h) == torch.Tensor:
    return Variable(h.data)
  else:
    d = tuple(repackage_hidden(v) for v in h)
    return d

In [0]:
def run_epoch(model, data, is_train=False, lr=1.0):
    """Runs the model on the given data."""
    if is_train:
        model.train()
    else:
        model.eval()
    epoch_size = ((len(data) // model.batch_size) - 1) // model.num_steps
    start_time = time.time()
    hidden = model.init_hidden()
    hidden[0].requires_grad=True
    hidden[1].requires_grad=True
    costs = 0.0
    iters = 0
    for step, (x, y) in enumerate(reader.ptb_iterator(data, model.batch_size, model.num_steps)):
        inputs =torch.from_numpy(x.astype(np.int64)).transpose(0, 1).contiguous().cuda()
        model.zero_grad()
        hidden = repackage_hidden(hidden)
        outputs, hidden = model(inputs, hidden)
        targets = torch.from_numpy(y.astype(np.int64)).transpose(0, 1).contiguous().cuda()
        tt = torch.squeeze(targets.view(-1, model.batch_size * model.num_steps))

        loss = criterion(outputs.view(-1, model.vocab_size), tt)
        #print( loss.data.item() , model.num_steps)
        costs += loss.data.item() * model.num_steps
        iters += model.num_steps

        if is_train:
            loss.backward()
            torch.nn.utils.clip_grad_norm(model.parameters(), 0.25)
            for p in model.parameters():
                p.data.add_(-lr, p.grad.data)
            if step % (epoch_size // 10) == 10:
                print("{} perplexity: {:8.2f} speed: {} wps".format(step * 1.0 / epoch_size, np.exp(costs / iters),
                                                       iters * model.batch_size / (time.time() - start_time)))
    return np.exp(costs / iters)


In [0]:
#for google collab
!mkdir /content/data

In [0]:

raw_data = reader.ptb_raw_data(data_path=data)
train_data, valid_data, test_data, word_to_id, id_2_word = raw_data
vocab_size = len(word_to_id)
print('Vocabluary size: {}'.format(vocab_size))
model = LM_LSTM(embedding_dim=hidden_size, num_steps=num_steps, batch_size=batch_size,
                vocab_size=vocab_size, num_layers=num_layers, dp_keep_prob=dp_keep_prob)
model.cuda()
lr = inital_lr
# decay factor for learning rate
lr_decay_base = 1 / 1.15
# we will not touch lr for the first m_flat_lr epochs
m_flat_lr = 14.0

print("########## Training ##########################")
for epoch in range(num_epochs):
    lr_decay = lr_decay_base ** max(epoch - m_flat_lr, 0)
    lr = lr * lr_decay # decay lr if it is time
    train_p = run_epoch(model, train_data, True, lr)
    print('Train perplexity at epoch {}: {:8.2f}'.format(epoch, train_p))
    print('Validation perplexity at epoch {}: {:8.2f}'.format(epoch, run_epoch(model, valid_data)))
print("########## Testing ##########################")
model.batch_size = 1 # to make sure we process all the data
print('Test Perplexity: {:8.2f}'.format(run_epoch(model, test_data)))
with open(save, 'wb') as f:
    torch.save(model, f)
print("########## Done! ##########################")

Vocabluary size: 10000
########## Training ##########################


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.


0.007535795026375283 perplexity: 11527.40 speed: 2844.6852155846427 wps
0.10700828937452901 perplexity:  1499.09 speed: 3110.6207065535705 wps
0.20648078372268275 perplexity:   990.26 speed: 3120.316663844005 wps
0.3059532780708365 perplexity:   770.46 speed: 3121.5829530410356 wps
0.4054257724189902 perplexity:   656.31 speed: 3121.103484107114 wps
0.5048982667671439 perplexity:   582.19 speed: 3120.606021969607 wps
0.6043707611152976 perplexity:   522.02 speed: 3120.8393492188093 wps
0.7038432554634514 perplexity:   479.49 speed: 3120.3037921147547 wps
0.8033157498116051 perplexity:   446.33 speed: 3120.6308103242695 wps
0.9027882441597589 perplexity:   417.47 speed: 3120.3670190784806 wps
Train perplexity at epoch 0:   395.04
Validation perplexity at epoch 0:   225.94
0.007535795026375283 perplexity:   278.42 speed: 3054.9332994318183 wps
0.10700828937452901 perplexity:   220.00 speed: 3123.216264339193 wps
0.20648078372268275 perplexity:   227.16 speed: 3126.2871508714006 wps
0.305

Загрузка обученной модели

In [15]:
model =  torch.load('./lm_model.pt')
model.cuda()
model.eval()

LM_LSTM(
  (dropout): Dropout(p=0.65)
  (word_embeddings): Embedding(10000, 1500)
  (lstm): LSTM(1500, 1500, num_layers=2, dropout=0.65)
  (sm_fc): Linear(in_features=1500, out_features=10000, bias=True)
)

In [18]:
len(train_data)

929589